In [436]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [437]:
# import gc

# # Delete the model
# del wv

# # Force garbage collection
# gc.collect()


In [438]:
# import gensim.downloader as api
# wv = api.load("word2vec-google-news-300")

# vec_king = wv["king"]

In [439]:
import pandas as pd
messages = pd.read_csv("smsspamcollection/SMSSpamCollection",sep = "\t", names = ["label","messages"])

In [440]:
# vec_king

In [441]:
from nltk.stem import WordNetLemmatizer

In [442]:
lem = WordNetLemmatizer()

In [443]:
## here for understanding purpose we are not going to remove stopwords
#just too check how basic words are converted
import re

In [444]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub("[^a-zA-z]"," ", messages["messages"][i])
    review = review.lower()
    review = review.split()
    # now we have each sentence as list of words

    review = [lem.lemmatize(word) for word in review]
    review = " ".join(review)
    ## forming the  sentence as list again
    corpus.append(review)

In [445]:
## later in code I discrovered that after I did the words array of array creation that 3 rows are dropping off somewhere
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['messages']) if i< 1]
## so in a way we are making 3 iterators and then iterating through 

#these three entries are missing because of the preprcessing steps
# we should remove these from y also

[[0, '', '645'], [0, '', ':) '], [0, '', ':-) :-)']]

In [446]:
len(words)

5569

In [447]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [448]:
words =[]
for sent in corpus:
    sent_token = sent_tokenize(sent)
    # froom aa parangraph firstly getting sentances 
    for word in sent_token:
        ## this second method can also be used to preprocess where in we can paass the sentence tokens to simple_preprocess 
        words.append(simple_preprocess(word))

In [449]:
# list words here would contain list of words in each  sentences
words[0]
# representation of one sentence

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

### using gensim from scratch

In [450]:
### word2vec from scratch
## earlier when we used word to vec we used the pretrained news acticle data toget rep , now using our data

In [451]:
model = gensim.models.Word2Vec(words)
# default feature size is 100
## can pass epochs etc here to make sure on how many epocs we tain this model on our data

In [452]:
model.corpus_count
## no of sentecnes  on which trained

5569

In [453]:
model.epochs

5

In [454]:
model.wv.most_similar('kid')
# n our vocabulary whixh are most similar words to wrod kid

[('stuff', 0.9975462555885315),
 ('down', 0.9975353479385376),
 ('ask', 0.9974292516708374),
 ('doe', 0.9974199533462524),
 ('could', 0.9974098205566406),
 ('haha', 0.9973984360694885),
 ('even', 0.9973711371421814),
 ('out', 0.9973620176315308),
 ('actually', 0.9973409175872803),
 ('amp', 0.9973403215408325)]

In [455]:
model.wv['good']

array([-0.29679525,  0.37631786,  0.18575299,  0.19054769,  0.07910077,
       -0.67115635,  0.26704744,  0.7039873 , -0.37193224, -0.1904544 ,
       -0.26920864, -0.5036599 , -0.04637603,  0.14259544,  0.30070046,
       -0.28751042,  0.20274957, -0.39988643, -0.13557966, -0.78346235,
        0.37224182,  0.10166611,  0.12815718, -0.24928622, -0.05463323,
       -0.0093995 , -0.27534997, -0.29339388, -0.44311234,  0.09213511,
        0.37698174,  0.02168418,  0.15909405, -0.32078823, -0.15048134,
        0.5157417 ,  0.0121179 , -0.17925806, -0.15051308, -0.6934408 ,
        0.09100346, -0.24837519, -0.24062788,  0.0405445 ,  0.23734446,
       -0.050428  , -0.16045621, -0.08290946,  0.3223107 ,  0.16926962,
        0.2035431 , -0.23147258, -0.09339802,  0.0132419 ,  0.01552789,
        0.10723267,  0.2182554 ,  0.11485488, -0.5024585 ,  0.2438796 ,
       -0.00272788,  0.19599174, -0.05493554, -0.1410245 , -0.33713752,
        0.37977546,  0.17153451,  0.31646812, -0.40324   ,  0.52

### using avd word2vec

In [456]:
# for every word we have 100 dim we want 100 dim for every sentence

In [457]:
import numpy as np
def avg_word2vec(doc):
    # so if in a doc we have words , for each word if that word in our vocabulary we would 
    # calculate the mean of words whhich are there ni that particular doc leaving the words which are not there
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis = 0)

In [458]:
## progress bar

In [459]:
from tqdm import tqdm

In [460]:
## aplly word2vec on each sentence:

In [461]:
X = []
for i in tqdm(range(0,len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5569 [00:00<?, ?it/s]C:\Users\thaku\anaconda3\envs\nlpx\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\thaku\anaconda3\envs\nlpx\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5569/5569 [00:00<00:00, 6375.62it/s]


In [462]:
len(X[100])

100

In [463]:
## complete sntence represented as 100 dim vec
len(X)

5569

In [464]:
X_new = np.array(X, dtype=object)

In [465]:
X_new.shape
len(corpus)

5572

In [466]:
messages[list(map(lambda x : len(x) >0,corpus))].shape

(5569, 2)

In [467]:
# filtering the entries from the data itself
y = messages[list(map(lambda x : len(x) > 0,corpus))]
y = pd.get_dummies(y['label'])

In [468]:
y.shape

(5569, 2)

In [469]:
y = y.iloc[:,0].values

In [470]:
messages.shape

(5572, 2)

In [471]:
y.shape

(5569,)

In [472]:
## shape of orginal data is 5572 and of the x_NEW IS 5569 some issue is there for sure

In [473]:
X[0].shape

(100,)

In [474]:
X[0].reshape(1,-1).shape

(1, 100)

In [480]:
dfs  = []
for i in range (0,len(X)):
    df = pd.DataFrame(X[i].reshape(1,-1))
    dfs.append(df)

In [481]:
df = pd.concat(dfs, ignore_index=True)

C:\Users\thaku\AppData\Local\Temp\ipykernel_19824\69934419.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, ignore_index=True)


In [482]:
type(dfs)

list

In [483]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.199557,0.262728,0.140453,0.126231,0.076066,-0.490201,0.169265,0.486570,-0.262596,-0.158662,...,0.382796,0.144859,0.059135,0.040921,0.482592,0.185473,0.135428,-0.241599,0.183057,0.007152
1,-0.182087,0.225032,0.119098,0.109012,0.073934,-0.420796,0.130697,0.422781,-0.225505,-0.132019,...,0.336122,0.115900,0.045137,0.029396,0.400762,0.153643,0.115656,-0.216398,0.162219,0.001012
2,-0.200557,0.290213,0.146184,0.143430,0.054120,-0.527835,0.166102,0.485733,-0.286185,-0.196777,...,0.370795,0.141367,0.033377,0.032064,0.501388,0.170128,0.082857,-0.272957,0.207053,0.038301
3,-0.281285,0.354928,0.188927,0.175514,0.107097,-0.665953,0.223738,0.669136,-0.356880,-0.208749,...,0.527517,0.193783,0.082619,0.060925,0.643466,0.255542,0.200216,-0.333914,0.251997,-0.001207
4,-0.241334,0.290233,0.165663,0.145448,0.097039,-0.558744,0.186864,0.565717,-0.302588,-0.178086,...,0.443885,0.161164,0.074178,0.053702,0.545903,0.216509,0.163407,-0.289041,0.209333,-0.002847


In [484]:
X =df

In [485]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =42)

In [486]:
y.shape

(5569,)

In [487]:
from sklearn.ensemble import RandomForestClassifier

In [488]:
classifier = RandomForestClassifier()

In [489]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [490]:
y_pred = classifier.predict(X_test)

In [492]:
from sklearn.metrics import classification_report, accuracy_score

In [493]:
print(accuracy_score(y_pred, y_test))

0.9757630161579892


In [494]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

       False       0.89      0.94      0.91       150
        True       0.99      0.98      0.99       964

    accuracy                           0.98      1114
   macro avg       0.94      0.96      0.95      1114
weighted avg       0.98      0.98      0.98      1114

